<h2> Multivariate Logistic Regression </h2>

<h3> Importing the data </h3>

Let's get started by loading the data set. Unlike the previous examples, our data file is in a format native to MATLAB and not automatically recognizable by pandas, so to load it in Python we need to use a SciPy utility (remember the data files are available at the link at the top of the post).

In [23]:
import numpy as np  
import pandas as pd  
import matplotlib.pyplot as plt  
from scipy.io import loadmat  
%matplotlib inline

data = loadmat('data/ex3data1.mat')  
data

{'X': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        ..., 
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ...,  0.,  0.,  0.]]),
 '__globals__': [],
 '__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 'y': array([[10],
        [10],
        [10],
        ..., 
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [24]:
data['X'].shape, data['y'].shape, data['X'].T.shape, data['y'].T.shape

((5000, 400), (5000, 1), (400, 5000), (1, 5000))

<h3> Sigmoid Function </h3>

In [25]:
def sigmoid(z):  
    return 1 / (1 + np.exp(-z))

<h3> Cost Function </h3>

In [26]:
def cost(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)
    first = np.multiply(-y, np.log(sigmoid(X * theta.T)))
    second = np.multiply((1 - y), np.log(1 - sigmoid(X * theta.T)))
    reg = (learningRate / 2 * len(X)) * np.sum(np.power(theta[:,1:theta.shape[1]], 2))
    return np.sum(first - second) / (len(X)) + reg

<h3> Gradient Descent Function </h3>

\begin{equation}
\theta_0 := \theta_0 * \alpha \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^i)-y^i) (x_0^i) \\
\theta_j := \theta_j * \alpha \frac{1}{m} \sum_{i=1}^{m} (h_\theta(x^i)-y^i) (x_j^i) + \frac\lambda {m} \theta_j
\end{equation}

<b> Loop Form </b>

In [27]:
def gradient_with_loop(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])
    grad = np.zeros(parameters)

    error = sigmoid(X * theta.T) - y

    for i in range(parameters):
        term = np.multiply(error, X[:,i])

        if (i == 0):
            grad[i] = np.sum(term) / len(X)
        else:
            grad[i] = (np.sum(term) / len(X)) + ((learningRate / len(X)) * theta[:,i])

    return grad

<b> Vectorized Form </b>

In [28]:
def gradient(theta, X, y, learningRate):  
    theta = np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    parameters = int(theta.ravel().shape[1])
    error = sigmoid(X * theta.T) - y

    grad = ((X.T * error) / len(X)).T + ((learningRate / len(X)) * theta)

    # intercept gradient is not regularized
    grad[0, 0] = np.sum(np.multiply(error, X[:,0])) / len(X)

    return np.array(grad).ravel()

Also note that we're converting the data structures to NumPy matrices (which I've used for the most part throughout these exercises). This is done in an attempt to make the code look more similar to Octave than it would using arrays because matrices automatically follow matrix operation rules vs. element-wise operations, which is the default for arrays. There is some debate in the community over whether or not the matrix class should be used at all, but it's there so we're using it in these examples.

<h3>What it's means matrix operation rules vs element-wise operations, make reserach on this area???</h3>

<h3> One-vs-all Classification </h3>

In [37]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):  
    rows = X.shape[0]
    params = X.shape[1]

    # k X (n + 1) array for the parameters of each of the k classifiers
    all_theta = np.zeros((num_labels, params + 1))

    # insert a column of ones at the beginning for the intercept term
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # labels are 1-indexed instead of 0-indexed
    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))

        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x

    return all_theta

In [38]:
rows = data['X'].shape[0]  
params = data['X'].shape[1]

all_theta = np.zeros((10, params + 1))

X = np.insert(data['X'], 0, values=np.ones(rows), axis=1)

theta = np.zeros(params + 1)

y_0 = np.array([1 if label == 0 else 0 for label in data['y']])  
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape

((5000, 401), (5000, 1), (401,), (10, 401))

In [39]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [40]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)  
all_theta

array([[ -3.70247932e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -2.24803601e-10,   2.31962906e-11,   0.00000000e+00],
       [ -8.96250751e-05,   0.00000000e+00,   0.00000000e+00, ...,
          7.26120862e-09,  -6.19965329e-10,   0.00000000e+00],
       [ -8.39553375e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -7.61695635e-10,   4.64917650e-11,   0.00000000e+00],
       ..., 
       [ -7.00832719e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -6.92009371e-10,   4.29241679e-11,   0.00000000e+00],
       [ -7.65188090e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -8.09503513e-10,   5.31058851e-11,   0.00000000e+00],
       [ -6.63412491e-05,   0.00000000e+00,   0.00000000e+00, ...,
         -3.49765984e-09,   1.13668571e-10,   0.00000000e+00]])

<h3> One-vs-all Prediction </h3>

In [33]:
def predict_all(X, all_theta):  
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]

    # same as before, insert ones to match the shape
    X = np.insert(X, 0, values=np.ones(rows), axis=1)

    # convert to matrices
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)

    # compute the class probability for each class on each training instance
    h = sigmoid(X * all_theta.T)

    # create array of the index with the maximum probability
    h_argmax = np.argmax(h, axis=1)

    # because our array was zero-indexed we need to add one for the true label prediction
    h_argmax = h_argmax + 1

    return h_argmax

In [35]:
y_pred = predict_all(data['X'], all_theta)  
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]  
accuracy = (sum(map(int, correct)) / float(len(correct)))  
print (accuracy * 100)

74.6
